In [186]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score

import numpy as np
import pandas as pd

In [173]:
def process_domain_with_copyright_col(x):
    if isinstance(x, str):
        x = x.lower()
        if x == "zero" or x == "0":
            return 0
        if x == "one" or x == "1":
            return 1
    else:
        return x

In [174]:
# Load the data and drop the url and status columns
df = pd.read_csv('./dataset_link_phishing.csv', low_memory=False)
data = df.iloc[:, 2:-1]
y = df.iloc[:, -1]

In [175]:
label_encoder = LabelEncoder()
data['domain_with_copyright'] = data['domain_with_copyright'].apply(process_domain_with_copyright_col)
print(data['domain_with_copyright'])

0        1
1        0
2        0
3        1
4        0
        ..
19426    0
19427    0
19428    0
19429    0
19430    1
Name: domain_with_copyright, Length: 19431, dtype: int64


In [176]:
# Splitting the dataset into training (80%), validation (10%), and test sets (10%)
x_train, x_temp, y_train, y_temp = train_test_split(data, y, test_size=0.2, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

In [183]:
# Note: without the scaler, the model could not be trained
pipeline = make_pipeline(StandardScaler(), LogisticRegression(max_iter=500))

# Train the mocel
pipeline.fit(x_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression(max_iter=500))])

In [184]:
# Validate the model
val_predictions = pipeline.predict(x_val)

val_accuracy = accuracy_score(y_val, val_predictions)
print("Validation Accuracy:", val_accuracy)

Validation Accuracy: 0.9485331960885229


In [185]:
# Test the model
test_predictions = pipeline.predict(x_test)

test_accuracy = accuracy_score(y_test, test_predictions)
print("Test Accuracy:", test_accuracy)

Test Accuracy: 0.9387860082304527


In [191]:
# Define a list of solvers to compare
solvers = ['lbfgs', 'saga', 'liblinear', 'newton-cg']

# Create a dictionary to store cross-validation scores for each solver
solver_scores = {}

# Iterate over each solver
for solver in solvers:
    # Create a pipeline with StandardScaler and logistic regression with the current solver
    pipeline = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, solver=solver))
    
    # Perform cross-validation and store the scores
    scores = cross_val_score(pipeline, x_train, y_train, cv=5)
    solver_scores[solver] = scores

# Print the cross-validation scores for each solver
for solver, scores in solver_scores.items():
    print(f"Solver: {solver}, Mean Accuracy: {scores.mean():.4f}, Std Dev: {scores.std():.4f}")

c:\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Solver: lbfgs, Mean Accuracy: 0.9481, Std Dev: 0.0015
Solver: saga, Mean Accuracy: 0.9481, Std Dev: 0.0018
Solver: liblinear, Mean Accuracy: 0.9483, Std Dev: 0.0015
Solver: newton-cg, Mean Accuracy: 0.9483, Std Dev: 0.0015
